In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

from sklearn.cluster import SpectralClustering

import pycombo

import pandas as pd

torch.set_printoptions(sci_mode=False)

import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from utils import doublerelu

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-10
epochs = 1000
seed = 165

In [4]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [5]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [7]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [8]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [9]:
def community_detection(G,method,GNN,features=None,hidden=1):
    
    adj = nx.to_numpy_matrix(G)
    adj = np.expand_dims(adj, axis=0)
        
    #Combo
    combo_partition = pycombo.execute(G)
    nb_comm = max(combo_partition[0].values())+1
    combo_mod = combo_partition[1]

    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj))
    adj_norm = torch.FloatTensor(np.array(adj_norm))
    
        
    if type(features) == type(None):

        # features
        if method == "louvain":
            partition = community_louvain.best_partition(G)
            #get binary matrix of the partition
            nb_community = max(list(partition.values())) + 1
            communities =  np.array(list(partition.values())).reshape(-1)
            C = np.eye(nb_community)[communities]
            C = torch.FloatTensor(C)
            #perturbations
            C = perturb(C)
        elif method == "spectral":
            partition = spectralPartition(adj,nb_comm)
            C = partition.float()
            nb_community = C.shape[1]

        features = torch.FloatTensor(C)
        features = features.unsqueeze(0)
    
    nb_community = features.shape[-1]
    
    if method == "louvain":
        lr = 0.001
    elif method == "spectral":
        lr = 0.001
        

    Q = modularity_matrix(adj)
    
    # Model and optimizer

    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_community,
                hidden=hidden)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        Q = Q.cuda()

    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    
    
    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        C = model(features, adj_norm)
        loss = modularity(C,Q)

        loss.backward()

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            init_mod = loss
            best_C = C
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return nb_community,-init_mod,-best_loss,combo_mod, best_C

In [10]:
def perturb(A):
    p = 0.4
    epsilon = torch.rand(A.shape).uniform_(0, p) - torch.rand(A.shape).uniform_(0, p)
    return torch.clip(A + epsilon,0,1)

In [11]:
def spectralPartition(A,nb_comm):
    A = A[0] # TODO 3D
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [12]:
G = [nx.karate_club_graph(),nx.les_miserables_graph(),nx.florentine_families_graph(),nx.house_graph()]
methods = ["louvain","spectral"]
nb_communities = []
init_mods = []
finetuned_mods = []
combo_mods = []
methodss = []
networks = ["Karate Club","Karate Club","Les Miserables","Les Miserables","Florentine","Florentine","House Graph","House Graph"]
for network in G:
    for method in methods:
        print(network,end=" ")
        print(method)
        nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=1)
        nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features.detach(),hidden=nb_community+4)
        initial_mod = torch.clone(init_mod)
        while(init_mod<finetuned_mod):
            nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features.detach(),hidden=nb_community+4)
        nb_communities.append(nb_community)
        init_mods.append(initial_mod.cpu().detach().numpy()[0][0])
        finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
        combo_mods.append(combo_mod)
        methodss.append(method)

Zachary's Karate Club louvain
Epoch: 0001 Modularity: 0.23050189 time: 0.3917s
Epoch: 0101 Modularity: 0.38391131 time: 0.0125s
Epoch: 0201 Modularity: 0.41260108 time: 0.0000s
Epoch: 0301 Modularity: 0.41529948 time: 0.0000s
Epoch: 0401 Modularity: 0.41559827 time: 0.0000s
Epoch: 0501 Modularity: 0.41559827 time: 0.0000s
Epoch: 0601 Modularity: 0.41559827 time: 0.0000s
Epoch: 0701 Modularity: 0.41559827 time: 0.0000s
Epoch: 0801 Modularity: 0.41559827 time: 0.0000s
Epoch: 0901 Modularity: 0.41559827 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.9634s
Epoch: 0001 Modularity: 0.41559827 time: 0.0125s
Epoch: 0101 Modularity: 0.41559827 time: 0.0000s
Epoch: 0201 Modularity: 0.41559827 time: 0.0000s
Epoch: 0301 Modularity: 0.41559827 time: 0.0000s
Epoch: 0401 Modularity: 0.41559827 time: 0.0000s
Epoch: 0501 Modularity: 0.41559827 time: 0.0000s
Epoch: 0601 Modularity: 0.41559827 time: 0.0110s
Epoch: 0701 Modularity: 0.41559827 time: 0.0156s
Epoch: 0801 Modularity: 0.41559827 ti

Epoch: 0101 Modularity: 0.35727596 time: 0.0156s
Epoch: 0201 Modularity: 0.39439148 time: 0.0000s
Epoch: 0301 Modularity: 0.39875001 time: 0.0000s
Epoch: 0401 Modularity: 0.39875001 time: 0.0156s
Epoch: 0501 Modularity: 0.39875001 time: 0.0000s
Epoch: 0601 Modularity: 0.39875001 time: 0.0000s
Epoch: 0701 Modularity: 0.39875001 time: 0.0000s
Epoch: 0801 Modularity: 0.39875001 time: 0.0156s
Epoch: 0901 Modularity: 0.39875001 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.4819s
Epoch: 0001 Modularity: 0.39875001 time: 0.0000s
Epoch: 0101 Modularity: 0.39875001 time: 0.0000s
Epoch: 0201 Modularity: 0.39875001 time: 0.0000s
Epoch: 0301 Modularity: 0.39875001 time: 0.0000s
Epoch: 0401 Modularity: 0.39875001 time: 0.0000s
Epoch: 0501 Modularity: 0.39875001 time: 0.0000s
Epoch: 0601 Modularity: 0.39875001 time: 0.0000s
Epoch: 0701 Modularity: 0.39875001 time: 0.0000s
Epoch: 0801 Modularity: 0.39875001 time: 0.0156s
Epoch: 0901 Modularity: 0.39875001 time: 0.0156s
Optimization Finis

In [13]:
df = pd.DataFrame([networks,methodss,nb_communities,init_mods,finetuned_mods,combo_mods]).transpose()

In [14]:
df.columns = ["Network","Method","Communities","Initial Modularity","Finetuned Moduclarity","Combo Modularity"]

In [15]:
df

,Network,Method,Communities,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,0.415598,0.415598,0.419790
1,Karate Club,spectral,4,0.365322,0.399080,0.419790
2,Les Miserables,louvain,6,0.565376,0.565416,0.566688
3,Les Miserables,spectral,6,0.390676,0.484918,0.566688
4,Florentine,louvain,3,0.398750,0.398750,0.398750
5,Florentine,spectral,3,0.300000,0.300000,0.398750
6,House Graph,louvain,2,0.111111,0.111111,0.111111
7,House Graph,spectral,2,-0.004795,0.111111,0.111111
